# <b>1. 환경설정하기</b>

- **Convolutional neural network(CNN)**를 사용하여 감정 분석을 할 예정입니다. 
- [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882) paper의 model을 사용하였습니다.

In [1]:
!pip install torchtext==0.6.0
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 573.0 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:00 MB/s eta 0:00:010:01
/opt/homebrew/Caskroom/miniforge/base/envs/torch/bin/python: No module named spacy


In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [3]:
nn.Conv2d?

Init signature:
nn.Conv2d(
    in_channels: int,
    out_channels: int,
    kernel_size: Union[int, Tuple[int, int]],
    stride: Union[int, Tuple[int, int]] = 1,
    padding: Union[str, int, Tuple[int, int]] = 0,
    dilation: Union[int, Tuple[int, int]] = 1,
    groups: int = 1,
    bias: bool = True,
    padding_mode: str = 'zeros',
    device=None,
    dtype=None,
) -> None
Docstring:     
Applies a 2D convolution over an input signal composed of several input
planes.

In the simplest case, the output value of the layer with input size
:math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
can be precisely described as:

.. math::
    \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
    \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


where :math:`\star` is the valid 2D `cross-correlation`_ operator,
:math:`N` is a batch size, :math:`C` denotes a number of channels,
:

In [4]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# <b>2. torchtext로 전처리하기</b>
- data는 IMDB dataset을 사용하겠습니다

In [9]:
!pip install sapcy

ERROR: Could not find a version that satisfies the requirement sapcy (from versions: none)
ERROR: No matching distribution found for sapcy


In [5]:
TEXT = data.Field(tokenize='spacy', 
                  tokenizer_language = 'en_core_web_sm', 
                  sequential=True, 
                  batch_first=True, 
                  lower=True)
LABEL = data.LabelField(sequential=False, 
                        batch_first=True, 
                        dtype=torch.long)

Please install SpaCy. See the docs at https://spacy.io for more information.


ModuleNotFoundError: No module named 'spacy'

## <b>(2) data로드 및 분할하기</b>
- torchtext.datasets으로 IMDB dataset을 다운로드 받고, train, test로 분할

In [ ]:
trn_dset, tst_dset = datasets.IMDB.splits(TEXT, LABEL)
trn_dset, val_dset = trn_dset.split(random_state = random.seed(seed), 
                                    split_ratio = 0.8)

In [6]:
print(len(trn_dset))
print(len(val_dset))
print(len(tst_dset))

NameError: name 'trn_dset' is not defined

## <b>(3) 단어 집합(vocab) 만들기
- glove.6B.100의 학습된 임베딩 벡터를 사용
    - (glove도 word2vec처럼 일종의 임베딩 방법론이다 라는 정도만 알고 넘어갑시다)

In [7]:
MAX_VOCAB_SIZE=25000
TEXT.build_vocab(trn_dset, 
                 max_size = MAX_VOCAB_SIZE,
                 vectors = 'glove.6B.100d', # 기존에 학습된 100차원의 벡터를 갖고오고
                 unk_init = torch.Tensor.normal_) # glove에서 학습되지 않은 벡터는 임의로 생성
LABEL.build_vocab(trn_dset)

NameError: name 'TEXT' is not defined

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

NameError: name 'TEXT' is not defined

In [ ]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fb959185f90>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             ',': 3,
             '.': 4,
             'and': 5,
             'a': 6,
             'of': 7,
             'to': 8,
             'is': 9,
             'it': 10,
             'in': 11,
             'i': 12,
             'this': 13,
             'that': 14,
             '"': 15,
             "'s": 16,
             '-': 17,
             '/><br': 18,
             'was': 19,
             'as': 20,
             'for': 21,
             'with': 22,
             'movie': 23,
             'but': 24,
             'film': 25,
             'you': 26,
             'on': 27,
             '(': 28,
             "n't": 29,
             ')': 30,
             'not': 31,
             'are': 32,
             'he': 33,
             'his': 34,
             'have': 35,
             'be': 36,
             'one': 37,
      

## <b>(4) data를 불러오기 위한 iterator 생성하기</b>
- torchtext.data.BucketIterator

In [ ]:
# Embedding vector관련
VOCAB_SIZE = len(TEXT.vocab)
EMBED_DIM = 100

# CNN구조 관련
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
N_CLASSES = 2
DROPOUT_RATIO = 0.3

# Train관련
BATCH_SIZE = 64
LR = 0.001
EPOCHS = 10
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [ ]:
trn_iter, val_iter, tst_iter = data.BucketIterator.splits(datasets = (trn_dset, val_dset, tst_dset), 
                                                          batch_size = BATCH_SIZE, 
                                                          shuffle=True, 
                                                          repeat=False)

# <b> 3. CNN model 구현하기</b>

## <b>(1) model class 구현하기</b>

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_filters, filter_sizes, output_dim, dropout_ratio, pad_idx):
        super(CNN, self).__init__()
        assert len(filter_sizes) == 3
        self.embed = nn.Embedding(n_vocab, embed_dim, padding_idx=pad_idx)
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels = n_filters, 
                               kernel_size=(filter_sizes[0], embed_dim))
        self.conv2 = nn.Conv2d(in_channels=1, 
                               out_channels=n_filters, 
                               kernel_size=(filter_sizes[1], embed_dim))
        self.conv3 = nn.Conv2d(in_channels=1, 
                               out_channels=n_filters, 
                               kernel_size=(filter_sizes[2], embed_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, text):
        # text size: [batch_size, sentence_len]
        embed = self.embed(text) # embed size: [batch_size, sentence_len, embed_dim]
        embed = embed.unsqueeze(1) # embed_size: [batch_size, 1, sentence_len, embed_dim]

        # conv(embed) size: (batch_size, n_filters, sentence_len-filter_size+1, 1)
        # squeezed size: (batch_size, n_filters, sentence_len-flter_size+1)
        conv_result1 = F.relu(self.conv1(embed).squeeze(3))
        conv_result2 = F.relu(self.conv2(embed).squeeze(3))
        conv_result3 = F.relu(self.conv3(embed).squeeze(3))

        # pool_result size: (batch_size, n_filters, 1)
        # squeezed size: (batch_size, n_filters)
        pool_result1 = F.max_pool1d(conv_result1, conv_result1.shape[2]).squeeze(2)
        pool_result2 = F.max_pool1d(conv_result2, conv_result2.shape[2]).squeeze(2)
        pool_result3 = F.max_pool1d(conv_result3, conv_result3.shape[2]).squeeze(2)

        # cat size: (batch_size, n_filters * len(filter_size))
        cat = torch.cat((pool_result1, pool_result2, pool_result3), dim=1)
        cat = self.dropout(cat)

        return self.fc(cat)
        

## <b>(2) 모델 객체 생성하기</b>

In [ ]:
model = CNN(n_vocab=VOCAB_SIZE, 
            embed_dim=EMBED_DIM, 
            n_filters=N_FILTERS, 
            filter_sizes=FILTER_SIZES, 
            output_dim=N_CLASSES, 
            dropout_ratio=DROPOUT_RATIO, 
            pad_idx=PAD_IDX)


## <b>(3) pretrained embeddings를 불러와서 model의 embedding vector에 대입</b>

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

In [ ]:
pretrained_embeddings

tensor([[ 1.9269,  1.4873,  0.9007,  ...,  0.1233,  0.3499,  0.6173],
        [ 0.7262,  0.0912, -0.3891,  ...,  0.0821,  0.4440, -0.7240],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4209,  0.9235,  0.1985,  ..., -0.7646,  0.1018,  0.3782],
        [ 0.1707,  0.2843, -0.0560,  ...,  0.0990, -0.2200,  0.2100],
        [ 0.1654,  0.0713, -0.0237,  ..., -0.4641,  0.7009,  0.6931]])

In [ ]:
pretrained_embeddings.shape

torch.Size([25002, 100])

In [ ]:
# 기존 model의 embedding vectors
model.embed.weight.data

tensor([[-1.5798, -0.9582, -0.3302,  ..., -0.3455,  0.4224, -1.3401],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4076,  1.2967,  0.3911,  ...,  0.5054,  0.3263,  0.9035],
        ...,
        [ 0.1310, -0.6473,  0.0078,  ..., -0.9512,  0.2684,  0.4572],
        [ 0.3971,  0.8289, -1.8827,  ..., -0.4697, -2.7504,  1.1220],
        [ 0.3028,  0.8413, -0.0300,  ..., -0.9295,  0.1378, -0.3617]])

In [ ]:
model.embed.weight.data.shape

torch.Size([25002, 100])

In [ ]:
# pretrained vectors로 기존 모델의 embedding vectors를 덮어씌운 결과
model.embed.weight.data.copy_(pretrained_embeddings)
model.embed.weight.data

tensor([[ 1.9269,  1.4873,  0.9007,  ...,  0.1233,  0.3499,  0.6173],
        [ 0.7262,  0.0912, -0.3891,  ...,  0.0821,  0.4440, -0.7240],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4209,  0.9235,  0.1985,  ..., -0.7646,  0.1018,  0.3782],
        [ 0.1707,  0.2843, -0.0560,  ...,  0.0990, -0.2200,  0.2100],
        [ 0.1654,  0.0713, -0.0237,  ..., -0.4641,  0.7009,  0.6931]])

In [ ]:
# Unknown(UNK) 토큰과 padding<PAD> token은 embedding weight를 0으로 초기화
# PAD_IDX=1(이미 위에서 구했음)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
UNK_IDX

0

In [ ]:
model.embed.weight.data[UNK_IDX] = torch.zeros(EMBED_DIM)
model.embed.weight.data[PAD_IDX] = torch.zeros(EMBED_DIM)

## <b>(4) optimizer 생성하기</b>

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR)

# <b>4. 모델 학습 및 평가 함수 생성하기</b>

In [ ]:
def train(model, opt, data_iterator):
    model.train()
    for i, batch in enumerate(data_iterator):
        x, y = batch.text.to(device), batch.label.to(device)
        opt.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, y)
        loss.backward()
        opt.step()

In [ ]:
def evaluate(model, data_iterator):
    model.eval()
    corrects, total_loss = 0, 0
    for _, batch in enumerate(data_iterator):
        x, y = batch.text.to(device), batch.label.to(device)
        output = model(x)
        loss = F.cross_entropy(output, y, reduction='sum')
        total_loss += loss.item()
        corrects += (output.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(data_iterator.dataset)
    total_loss /= size
    accuracy = 100*corrects / size
    return total_loss, accuracy

In [ ]:
best_val_loss = None
for epoch in range(1, EPOCHS+1):
    train(model, optimizer, trn_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print(f"[Epoch: {epoch}] val loss: {val_loss:.2f} | val accuracy: {val_accuracy:.2f}")

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/cnn_txtclassification.pt')
        best_val_loss = val_loss

KeyboardInterrupt: ignored